# chain of responsibility

## Pointer Chain

In [2]:
# imagine we have a character like in the game
# This charater will go through a series of modifications

from dataclasses import dataclass

@dataclass
class Player:
    name: str
    health: int = 100
    power: int = 100

    def __str__(self):
        return f"Player {self.name} has health {self.health} and power {self.power}"

In [1]:
# root modifier

from abc import abstractmethod

class PlayerModifier:
    
    _next = None

    def __init__(self, player):
        self.player = player

    def add(self, modifier):
        if self._next is not None:
            self._next.add(modifier)
        else:
            self._next = modifier
        return self._next

    @abstractmethod
    def modify(self):
        if self._next is not None:
            self._next.modify()

In [27]:
# concret modifiers

class HealthPotion(PlayerModifier):
    def modify(self):
        self.player.health += 10
        super().modify()

class DoubleAttack(PlayerModifier):
    def modify(self):
        self.player.power *= 2
        super().modify()

class FallTrap(PlayerModifier):
    def modify(self):
        self.player.health -= 20
        super().modify()

In [28]:
# usage

player = Player("ninja")

playermodifier = PlayerModifier(player)

healthpotion = HealthPotion(player)
doubleattack = DoubleAttack(player)
falltrap = FallTrap(player)

playermodifier.add(healthpotion).add(doubleattack).add(falltrap)

playermodifier.modify()

print(player)

<super: <class 'HealthPotion'>, <HealthPotion object>>
Player ninja has health 90 and power 200


## Broker Chain

The aforementioned structure is very artificial and can't really be used in reality since rarely the scenerio happens in a prefixed order.

In [72]:
import abc

# event broker (observer)
# this part can be remplaced by a dedicated library

class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

class What2Query(Enum):
    Attack = 1
    Heal = 2

In [93]:
# query and query handler

@dataclass
class Query:
    creature_name: str
    what_to_query: What2Query
    value: int
    
class Game:
    def __init__(self):
        self.queries = Event()
        
    def perform_query(self, query):
        self.queries(query)

In [94]:
# component implemenation

class Player:
    def __init__(self, game, name, health, power):
        self.game = game
        self.name = name
        self.health = health
        self.power = power
        
    @property
    def heal(self):
        q = Query(self.name, What2Query.Heal, self.health)
        self.game.perform_query(q)
        return q.value

    @property
    def attack(self):
        q = Query(self.name, What2Query.Attack, self.power)
        self.game.perform_query(q)
        return q.value

    def __str__(self):
        return f"{self.name}: {self.heal} / {self.attack}"

In [95]:
# modifiers

class PlayerModifier(abc.ABC):
    def __init__(self, game, player):
        self.game = game 
        self.player = player
        self.game.queries.append(self.handle)

    def handle(self, query):
        pass

class HealthPotion(PlayerModifier):
    def handle(self, query):
        if query.what_to_query == What2Query.Heal:
            query.value += 10
        
class DoubleAttack(PlayerModifier):
    def handle(self, query):
        if query.what_to_query == What2Query.Attack:
            query.value *= 2

In [96]:
# sample usage

game = Game()
player = Player(game, "ninja", 100, 100)
print(player)
HealthPotion(game, player)
DoubleAttack(game, player)
print(player)

ninja: 100 / 100
ninja: 110 / 200
